<h1>Clustering Toronto City, Canada  Neighborhoods</h1>

<h2> Import data from webiste, clean the data, insert location data and show on map</h2>

Import Required Libraries

In [1]:
import pandas as pd 
import geocoder # to get latitude and longitude values
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans # import k-means from clustering stage
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\etisalat\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.5                |   py38h32f6830_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  conda                                4.8.4-py38h32f6830_2 --> 4.8.5-py38h32f6830_0




conda-4.8.5          | 3.1 MB    |            |   0% 
conda-4.8.5          | 3.1 MB    |            |   1% 
conda-4.8.5          | 3.1 MB    | ###1       |  32% 
conda-4.8.5          | 3.1 MB    | #######1   |  71% 
conda-4.8.5          | 3.1 MB    | ########## | 100% 
conda-4.8.5          | 3.1 MB    | ########## | 100% 
Preparing transaction: ...working

<ol>
    <li>Read Canada's Postal code table from the url</li>
    <li>Remove any unassigned Borough</li>
<li>Replace any unassigned Neighbourhood with Borough name</li>
<li>Any repeated postal code is merged and Neighborhoods are combined with a ","</li>
 </ol>

In [2]:
#Read Canada's Postal code table from the url
df_Canada=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df_Canada=df_Canada.rename(columns={"Neighbourhood": "Neighborhood", "Community": "Borough"})
#Remove any unassigned Borough
df_Canada=df_Canada[df_Canada.Borough != 'Not assigned'] # removing unassigned borough
df_Canada.reset_index(drop=True)

#Replace unassigned neighbourhood with borough name
i=0    
while(i<103):
    if (df_Canada.iloc[i,2]=="Not assigned"):
        df_Canada.iloc[i,2]=df_Canada.iloc[i,1]    
    i=i+1
df_Canada.reset_index(drop=True)

#Any repeated postal code is merged and Neighborhoods are combined with a ","
df_Canada=df_Canada.groupby(['Postal Code','Borough'])['Neighborhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
df_Canada

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


Option 1: Adding lat and long to the Canada postal code dataframe using geocoder

In [3]:
lat_long=None
lat=[]
long=[]
df_Canada2=df_Canada.copy()
postal_code=df_Canada2['Postal Code'].tolist()
for code in postal_code:
    while(lat_long==None):
        g=geocoder.arcgis('{},Toronto,Ontario'.format(code))
        lat_long=g.latlng
    lat.append(lat_long[0])
    long.append(lat_long[1])
    lat_long=None
df_Canada2["Latitude"]=lat
df_Canada2["Longitude"]=long
df_Canada2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944
...,...,...,...,...,...
98,M9N,York,Weston,43.70357,-79.51645
99,M9P,Etobicoke,Westmount,43.69623,-79.52926
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.68674,-79.55729
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.74453,-79.58624


Option 2: Read lat & long csv file provided in coursera then merge it with Canada Postal code dataframe

In [4]:
df_geocode=pd.read_csv("https://cocl.us/Geospatial_data")
df_Canada = pd.merge(df_Canada,df_geocode, on='Postal Code')
df_Canada

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


Get lat & lng coord for Toronto City in Canada

In [5]:
address = 'Toronto,Canada'
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitudeee = location.latitude
longitudeee = location.longitude
print('The geograpical coordinates of Canada are {}, {}.'.format(latitudeee, longitudeee))

The geograpical coordinates of Canada are 43.6534817, -79.3839347.


Show a map centered around toronto city, Canada, with red marks showing it's Neighborhoods and Boroughs

In [6]:
# create map of New York using latitude and longitude values
map_toronto= folium.Map(location=[latitudeee, longitudeee], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighborhood in zip(df_Canada['Latitude'], df_Canada['Longitude'], df_Canada['Borough'], df_Canada['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
       [lat, lng],
       radius=5,
       popup=label,
       color='red',
       fill=True,
        fill_color='red',
       fill_opacity=0.7,
       parse_html=False
        ).add_to(map_toronto)  
    
map_toronto


<h2>Focus on and analyze Boroughs that contain the word Toronto </h2>

Create a dataframe with Boroughs that contain the word Toronto 

In [7]:
array=["East Toronto","West Toronto","Central Toronto","Downtown Toronto"]
df_Toronto = df_Canada[df_Canada['Borough'].isin(array)].reset_index(drop=True)
df_Toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


Show a map focused on Toronto city with blue markers pointing at Neighborhoods that are in Boroughs which contain Toronto word

In [8]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitudeee, longitudeee], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h2>Use Foursquare API to explore and analyze neighborhood in Boroughs that contain the word Toronto</h2>


Connect to Foursquare API using Client ID and Client Secret

In [9]:
CLIENT_ID = 'PTYI3W1TEN4UXF242ICMVB11MLF5UILKMEDH0WJL5VAIABZW' # your Foursquare ID
CLIENT_SECRET = 'PGGXVZO0A5HTPIAV1MWO0BP4TKY4WLAX5PMPDA1JL4CXL1YC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PTYI3W1TEN4UXF242ICMVB11MLF5UILKMEDH0WJL5VAIABZW
CLIENT_SECRET:PGGXVZO0A5HTPIAV1MWO0BP4TKY4WLAX5PMPDA1JL4CXL1YC


Define a function to get nearby venues of desired areas (Explore areas)

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
             # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

Call the function defined above to get nearby venues of Neighborhoods in Boroughs that contain the word Toronto 
Explore Neighborhoods in boroughs that contain the word Toronto 

In [11]:
LIMIT=100
toronto_venues = getNearbyVenues(names=df_Toronto['Neighborhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

Display the top 5 rows of the Dataframe containing nearby venues of Neighborhoods in Boroughs that contain the word Toronto 

In [12]:
print(toronto_venues.shape)
toronto_venues.head()

(1635, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Group the dataframe above by Neighborhood and get the count of venue per Neighborhood

In [13]:
toronto_num_venues=toronto_venues.groupby('Neighborhood').count()
toronto_num_venues

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,71,71,71,71,71,71
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


Show the number of unique venue categories and display the categories

In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues['Venue Category']

There are 237 uniques categories.


0                      Trail
1          Health Food Store
2                        Pub
3               Neighborhood
4           Greek Restaurant
                ...         
1630      Light Rail Station
1631                    Park
1632                 Butcher
1633    Gym / Fitness Center
1634        Recording Studio
Name: Venue Category, Length: 1635, dtype: object

Show map focused on Toronto city, with blue markers to show nearby venues of each Neighborhood in Boroughs that contain the word Toronto
Only first 200 were displayed

In [35]:
toronto_venues_map = folium.Map(location=[latitudeee, longitudeee], zoom_start=11) # generate map centred around Ecco
venue_toronto=toronto_venues.head(200)
for lat, lng, label in zip(venue_toronto['Venue Latitude'], venue_toronto['Venue Longitude'], venue_toronto['Venue Category']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(toronto_venues_map)
toronto_venues_map

Categorical data may be converted to numbers.
Use one hot encoding to convert categorical data to binary

In [16]:
# one hot encoding
toronto_onehotenco = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehotenco["Neighborhood"]=toronto_venues["Neighborhood"]
# move neighborhood column to the first column
neighbd=toronto_onehotenco["Neighborhood"]
toronto_onehotenco.drop(labels=['Neighborhood'], axis=1, inplace = True)
toronto_onehotenco.insert(0, 'Neighborhood', neighbd)
print(toronto_onehotenco.shape)
toronto_onehotenco

(1635, 237)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1631,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1632,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1633,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the rows of the above dataframe by Neighborhood and by taking the mean of the frequency of occurrence of each category

In [17]:
toronto_grouped = toronto_onehotenco.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped


(39, 237)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.014085,0.000000,0.000000,0.014085,0.00,0.014085
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.012987,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.012987,0.000000,...,0.012987,0.000000,0.000000,0.00,0.000000,0.000000,0.012987,0.000000,0.00,0.025974
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.040000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.00,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


Print each neighborhood along with the top 10 most common venue categories

In [18]:
num_top_venues = 10

for neigh in toronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                         venue  freq
0                  Coffee Shop  0.09
1               Farmers Market  0.04
2           Seafood Restaurant  0.04
3                       Bakery  0.04
4                  Cheese Shop  0.04
5                 Cocktail Bar  0.04
6                   Restaurant  0.04
7                     Beer Bar  0.04
8                 Concert Hall  0.02
9  Eastern European Restaurant  0.02


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.13
1            Coffee Shop  0.09
2              Nightclub  0.09
3         Breakfast Spot  0.09
4                    Gym  0.04
5     Italian Restaurant  0.04
6              Pet Store  0.04
7  Performing Arts Venue  0.04
8           Climbing Gym  0.04
9             Restaurant  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0                  Park  0.06
1         Auto Worksho

Convert the above to a dataframe
1. Define a function which sorts the venue categories in descending order.
2. Create the new dataframe and display the top 10 venue catogries for each neighborhood.

In [19]:
def sort_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = sort_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Bakery,Cocktail Bar,Beer Bar,Cheese Shop,Seafood Restaurant,Restaurant,Grocery Store,Italian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
2,"Business reply mail Processing Centre, South C...",Pizza Place,Auto Workshop,Skate Park,Light Rail Station,Park,Gym / Fitness Center,Comic Shop,Restaurant,Garden Center,Garden
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boutique,Plane,Airport,Airport Food Court,Airport Gate,Airport Terminal,Bar,Harbor / Marina
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Juice Bar,Department Store,Japanese Restaurant
5,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Restaurant,Baby Store,Candy Store,Nightclub,Coffee Shop
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Mediterranean Restaurant,Hotel,Café,Yoga Studio,Bubble Tea Shop
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Gym,American Restaurant,Seafood Restaurant,Gastropub,Japanese Restaurant,Bakery
8,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Café,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Restaurant,Indian Restaurant
9,Davisville North,Sandwich Place,Dog Run,Dance Studio,Hotel,Food & Drink Shop,Department Store,Gym / Fitness Center,Breakfast Spot,Park,Donut Shop


Define a functoin which returns the trending venues at that time of the desired areas

In [21]:
def gettrendingVenues(names, latitudes, longitudes):
    n=["No trending venues are available at the moment!","-","-","No trending venues are available at the moment!"]
    trending_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng, 
            VERSION)
             # make the GET request
        results = requests.get(url).json()["response"]['venues']
        if (results==[]):
            results=n[:]
            trending_list.append([(
            name, 
            lat, 
            lng, 
            results[0], 
            results[1], 
            results[2],  
            results[3])])

        else:
            # return only relevant information for each nearby venue
            trending_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    trending_venues = pd.DataFrame([item for trending_list in trending_list for item in trending_list])
    trending_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                   'Venue', 
                   'Venue Latitude', 
                   'Venue Longitude', 
                   'Venue Category']
    return(trending_venues)

Call the function above to get the trending venues(at that time) of neighborhoods in boroughs that contain the word Toronto

In [22]:
toronto_trending = gettrendingVenues(names=df_Toronto['Neighborhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

Show the dataframe of trending venues

In [23]:
print(toronto_trending.shape)
toronto_trending

(39, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!
1,"The Danforth West, Riverdale",43.679557,-79.352188,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!
2,"India Bazaar, The Beaches West",43.668999,-79.315572,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!
3,Studio District,43.659526,-79.340923,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!
4,Lawrence Park,43.728020,-79.388790,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!
5,Davisville North,43.712751,-79.390197,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!
6,"North Toronto West, Lawrence Park",43.715383,-79.405678,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!
7,Davisville,43.704324,-79.388790,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!
8,"Moore Park, Summerhill East",43.689574,-79.383160,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!
9,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,No trending venues are available at the moment!,-,-,No trending venues are available at the moment!


<h2>Cluster the neighborhoods in Toronto City (only in Boroughs that contain the word Toronto)</h2>

First of all, define the number of cluster(3 in this case) then run *k*-means to cluster the neighborhood into 3 clusters.

In [24]:
# set number of clusters
kclusters = 3

Toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Create a new dataframe that includes the cluster as well as the top 10 venue categories for each neighborhood.

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = df_Toronto.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Health Food Store,Trail,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Brewery,Pizza Place,Ice Cream Shop,Fish & Chips Shop,Steakhouse,Pub,Sushi Restaurant,Italian Restaurant,Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Bookstore,Seafood Restaurant,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Swim School,Bus Line,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Show a map centered around Toronto City with markers to show neighborhoods clustered into 5 clusters

In [26]:
# create map
map_clusters = folium.Map(location=[latitudeee, longitudeee], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Define dataframe of each cluster to examine and determine the discriminating venue categories that distinguish each cluster.

In [27]:
clusters={}
for i in range (0,6):
    clusters["cluster"+str(i)]=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == i, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

Cluster 1 Dataframe

In [28]:
clusters["cluster0"]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Pub,Health Food Store,Trail,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
1,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
2,"India Bazaar, The Beaches West",0,Park,Brewery,Pizza Place,Ice Cream Shop,Fish & Chips Shop,Steakhouse,Pub,Sushi Restaurant,Italian Restaurant,Restaurant
3,Studio District,0,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Bookstore,Seafood Restaurant,Cheese Shop
5,Davisville North,0,Sandwich Place,Dog Run,Dance Studio,Hotel,Food & Drink Shop,Department Store,Gym / Fitness Center,Breakfast Spot,Park,Donut Shop
6,"North Toronto West, Lawrence Park",0,Coffee Shop,Clothing Store,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant
7,Davisville,0,Dessert Shop,Sandwich Place,Pizza Place,Café,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Restaurant,Indian Restaurant
9,"Summerhill West, Rathnelly, South Hill, Forest...",0,Coffee Shop,Supermarket,American Restaurant,Sports Bar,Bank,Sushi Restaurant,Fried Chicken Joint,Restaurant,Bagel Shop,Pizza Place
11,"St. James Town, Cabbagetown",0,Coffee Shop,Restaurant,Pizza Place,Café,Bakery,Park,Italian Restaurant,Pub,Chinese Restaurant,Market
12,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Mediterranean Restaurant,Hotel,Café,Yoga Studio,Bubble Tea Shop


Cluster 2 Dataframe

In [29]:
clusters["cluster1"]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,1,Park,Swim School,Bus Line,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
8,"Moore Park, Summerhill East",1,Playground,Trail,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
10,Rosedale,1,Park,Playground,Trail,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
23,"Forest Hill North & West, Forest Hill Road Park",1,Sushi Restaurant,Park,Jewelry Store,Trail,Yoga Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Cluster 3 Dataframe

In [30]:
clusters["cluster2"]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,2,Garden,Music Venue,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
